In [54]:
from mosqito.sq_metrics import sharpness_din_tv, loudness_ecma, roughness_ecma, pr_ecma_st, tnr_ecma_st, pr_ecma_st, tnr_ecma_freq
from mosqito.sound_level_meter import comp_spectrum
import numpy as np
import librosa

In [83]:
audio, sr = librosa.load("audiofiles/1132730_13.wav", sr=48000, mono=True)

# N, N_time, N_spec, bark_axis, time_array = loudness_ecma(audio, sr) # N_time
# S, S_time = sharpness_din_tv(audio, fs=sr) # S
R, R_time, R_spec, bark_axis, time_axis = roughness_ecma(audio, sr) # R_time
    

/Users/matthiaserdmann/Documents/SoftwareDev/WebDev/soundscape-search/.venv/lib/python3.11/site-packages/mosqito/sq_metrics/roughness/roughness_ecma/_estimate_fund_mod_rate.py:43: RuntimeWarning: divide by zero encountered in divide
  crit = np.abs( (f_p_temp[ic] / (R_i0[ic] * f_p[i0])) - 1)


#### Calculate calibration factor

In [102]:
import numpy as np
import soundfile as sf

P_REF = 20e-6 

def sound_pressure(audio_data):
    p_rms = np.sqrt(np.mean(audio_data**2))  # RMS berechnen
    laeq = 20 * np.log10(p_rms / P_REF)  # Umrechnung in dB SPL
    return laeq

def calibration_value(audio_data, expected_l_ch1, expected_l_ch2):
    raw_laeg_ch1 = sound_pressure(audio_data[:, 0])
    raw_laeg_ch2 = sound_pressure(audio_data[:, 1])

    cal_ch1 = expected_l_ch1 - raw_laeg_ch1
    cal_ch2 = expected_l_ch2 - raw_laeg_ch2

    print(f"Kalibrierung für Datei:")
    print(f"  Kanal 1: Roh-LAeq={raw_laeg_ch1:.2f} dB → Erwartet={expected_l_ch1:.2f} dB → Kalibrierung={cal_ch1:.2f} dB")
    print(f"  Kanal 2: Roh-LAeq={raw_laeg_ch2:.2f} dB → Erwartet={expected_l_ch2:.2f} dB → Kalibrierung={cal_ch2:.2f} dB")

    return cal_ch1, cal_ch2

def apply_calibration(audio_data, cal_ch1, cal_ch2):
    audio_calibrated = np.zeros_like(audio_data)
    audio_calibrated[:, 0] = audio_data[:, 0] * (10**(cal_ch1 / 20))  # dB-Wert in linear gain Faktor und multipliziern
    audio_calibrated[:, 1] = audio_data[:, 1] * (10**(cal_ch2 / 20)) 
    return audio_calibrated


# Audiosignal laden
file_path = "../dataset/audiofiles/1132730_16.wav"
audio_data, fs = sf.read(file_path)

# kallibrieren
cal_ch1, cal_ch2 = calibration_value(audio_data, 64.13, 64.57)


Kalibrierung für Datei:
  Kanal 1: Roh-LAeq=65.01 dB → Erwartet=64.13 dB → Kalibrierung=-0.88 dB
  Kanal 2: Roh-LAeq=65.60 dB → Erwartet=64.57 dB → Kalibrierung=-1.03 dB


In [ ]:
file_path = "../dataset/audiofiles/1132730_13.wav"
audio_data, fs = sf.read(file_path)

print(cal_ch1)
print(cal_ch2)

calibrated_audio = apply_calibration(audio_data, cal_ch1, cal_ch2)

print(sound_pressure(calibrated_audio[:, 0]))
print(sound_pressure(calibrated_audio[:, 1]))

-0.8754655044905633
-1.0310607394430775
60.06171146898053
59.629264154569896


#### Pipeline

In [226]:
# load dataset
import pandas as pd
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client['soundscape_search']
collection = db['dataset_demo_temporalFeatures']

path = '../dataset/dataset.csv'
df = pd.read_csv(path)

if 'temporal_audio_features' not in df.columns:
    df['temporal_audio_features'] = None

In [227]:
# compute audio features and save to dataset
import os

def compute_acoustic_features(audio, sr):

    N, N_time, N_spec, bark_axis, time_array = loudness_ecma(audio, sr) # N_time
    S, S_time = sharpness_din_tv(audio, fs=sr) # S
    R, R_time, R_spec, bark_axis, time_axis = roughness_ecma(audio, sr) # R_time
    
    return N_time, S, R_time

def scale_acoustic_features_to_audio_length(feature_array, audio_length):

    original_length = len(feature_array)
    block_size = original_length / audio_length
    result = np.zeros(audio_length)
    
    for i in range(audio_length):
        start_idx = int(i * block_size)
        end_idx = int(min((i + 1) * block_size, original_length))
        
        # Average the values in this block
        if start_idx < end_idx:
            result[i] = np.mean(feature_array[start_idx:end_idx])
        else:
            result[i] = feature_array[start_idx] if start_idx < original_length else 0
            
    return result

folder_path = '../dataset/audiofiles'
audio_files = [f for f in os.listdir(folder_path) if f.endswith(('.wav'))]
    
    # Process each audio file
for audio_file in audio_files:
    file_path = os.path.join(folder_path, audio_file)
    
    try:
        # Load audio
        audio_data, sr = sf.read(file_path)

        # calibrate audio and convert to mono
        calibrated_audio = apply_calibration(audio_data, cal_ch1, cal_ch2)
        mono_audio = np.mean(calibrated_audio, axis=1)

        # compute acoustic features
        N_time, S, R_time = compute_acoustic_features(mono_audio, sr)

        # averagte features to audio length
        time_scaling_factor = 1
        length_audio = np.floor(len(mono_audio)/sr) + 1
        loudness = scale_acoustic_features_to_audio_length(N_time, int((length_audio*time_scaling_factor)))
        sharpness = scale_acoustic_features_to_audio_length(S, int((length_audio*time_scaling_factor)))
        roughness = scale_acoustic_features_to_audio_length(R_time, int((length_audio*time_scaling_factor)))

        audio_features = {
            "loudness": loudness.tolist(),
            "sharpness": sharpness.tolist(),
            "roughness": roughness.tolist()
        }

        # push to dataset
        file_name = file_path.split('/')[-1].split('.')[0]
        row_idx = df[df['file_name'] == file_name].index.to_list()

        if row_idx:
            df.at[row_idx[0], 'temporal_audio_features'] = audio_features
            print('\n done processing file ', audio_file)
            print('\n')
        else:
            print(f"No row found with file_name: {file_name}")
    except Exception as e:
        print(f"Error processing {audio_file}: {str(e)}")


[Warning] Signal resampled to 48 kHz fulfill the standard requirements and allow calculation.
[Warning] Signal resampled to 48 kHz to allow calculation. To fulfill the standard requirements fs should be >=48 kHz.
[Warning] when computing sharpness from time-varying loudness, a transient effect appears on the first points. To cut it, use 'skip='
[Warning] Signal resampled to 48 kHz fulfill the standard requirements and allow calculation.


/Users/matthiaserdmann/Documents/SoftwareDev/WebDev/soundscape-search/.venv/lib/python3.11/site-packages/mosqito/sq_metrics/roughness/roughness_ecma/_estimate_fund_mod_rate.py:43: RuntimeWarning: divide by zero encountered in divide
  crit = np.abs( (f_p_temp[ic] / (R_i0[ic] * f_p[i0])) - 1)



 done processing file  9754782_112.wav


[Warning] Signal resampled to 48 kHz fulfill the standard requirements and allow calculation.
[Warning] Signal resampled to 48 kHz to allow calculation. To fulfill the standard requirements fs should be >=48 kHz.
[Warning] when computing sharpness from time-varying loudness, a transient effect appears on the first points. To cut it, use 'skip='


/Users/matthiaserdmann/Documents/SoftwareDev/WebDev/soundscape-search/.venv/lib/python3.11/site-packages/mosqito/sq_metrics/sharpness/sharpness_din/sharpness_din_from_loudness.py:140: RuntimeWarning: divide by zero encountered in divide
  S = 0.11 * sum(N_specific * g * z * 0.1, axis=0) / N


[Warning] Signal resampled to 48 kHz fulfill the standard requirements and allow calculation.

 done processing file  1132730_20.wav


[Warning] Signal resampled to 48 kHz fulfill the standard requirements and allow calculation.
[Warning] Signal resampled to 48 kHz to allow calculation. To fulfill the standard requirements fs should be >=48 kHz.
[Warning] when computing sharpness from time-varying loudness, a transient effect appears on the first points. To cut it, use 'skip='
[Warning] Signal resampled to 48 kHz fulfill the standard requirements and allow calculation.

 done processing file  6867815_76.wav


[Warning] Signal resampled to 48 kHz fulfill the standard requirements and allow calculation.
[Warning] Signal resampled to 48 kHz to allow calculation. To fulfill the standard requirements fs should be >=48 kHz.
[Warning] when computing sharpness from time-varying loudness, a transient effect appears on the first points. To cut it, use 'skip='
[Warning] Signal resampled to 48 kHz

In [228]:
df.head()

,file_name,suffix,duration_s,pleasant,vibrant,eventful,chaotic,annoying,monotonous,uneventful,calm,ISO_Pleasantness,ISO_Eventfulness,SC_Nature,SC_Human,SC_Household,SC_Installation,SC_Signals,SC_Traffic,SC_Speech,SC_Music,Activity,Location8,FGsource,LAeq_default,N5_default,FavgArith_default,RAavgArith,SavgArith_default,R_default,T_default,temporal_audio_features
0,1132730_16,.wav,15.0,1,3,3,3,3,3,2,0,1.146447,2.646447,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,1_Cooking_housework_workout,3_Bathroom,Wäschetrockner,56.00,11.30,0.01880,7.390,1.06,0.1960,0.4650,"{'loudness': [6.742615349715177, 6.94395837086..."
1,1132730_32,.wav,15.0,1,1,2,3,3,2,0,1,1.146447,2.560660,0.0,0.0,3.6,0.0,0.0,0.0,0.0,0.0,4_Sleeping_waking_up_relaxing,1_LivingRoom,Heizlüfter 2 Stück,40.12,3.51,0.00840,2.620,1.14,0.0624,0.0401,"{'loudness': [3.2388867853512933, 3.2670160928..."
2,1134651_42,.wav,15.0,0,3,4,4,4,0,1,0,1.025126,3.646447,0.8,0.0,0.8,0.0,0.0,4.0,0.0,0.0,2_Concentrated_mental_work,1_LivingRoom,Baustelle,42.46,5.05,0.03400,6.310,1.23,0.0816,0.0949,"{'loudness': [2.4510143533583277, 3.1310662547..."
3,1996384_28,.wav,15.0,0,3,2,3,3,3,2,1,1.085786,2.292893,2.4,0.0,0.0,0.0,0.0,4.0,0.0,0.0,3_Social_interaction,1_LivingRoom,Verkehr,42.94,5.80,0.00740,2.890,1.36,0.0455,0.0594,"{'loudness': [2.4189740079283766, 2.4395130635..."
4,1132730_13,.wav,15.0,0,3,3,4,4,0,0,0,1.025126,3.646447,0.0,0.0,3.6,0.0,0.0,0.0,0.0,0.0,2_Concentrated_mental_work,2_Kitchen,Tiefkühlschrank,33.07,1.93,0.00623,0.999,1.12,0.2390,0.4260,"{'loudness': [1.6718705825136104, 1.8994561292..."


In [229]:
for index, soundscape in df.iterrows():
    collection.insert_one(soundscape.to_dict())